## **IMPLEMENTACIÓN DEL SISTEMA DE RECOMENDACIÓN**

#### **CARGA DE DATOS**

 ##### Durante el proceso ETL por descuido se eliminó la columna `overview`, por ende para recuperarla se crea un nuevo DataFrame, el mismo surge de la unión del dataset original y del dataset finalmente procesado y se realizan unas pequeñas transformaciones antes del merge:
- Del dataset original solo se extrae la columna `overview` previamente eliminada en el proceso ETL
- En el dataset procesado se conviertieron los `Id` a int comprobando si existían valores no númericos antes de la conversión

In [1]:
import pandas as pd

In [2]:
# dataset original el cual contiene la columna 'overview'
dataset_original_df = pd.read_csv('C:\DYNAMO\DATA SCIENCE\Proyectos\Proyecto-MLOps\Dataset\movies_dataset.csv', usecols=['id', 'overview'])

# dataset procesado
dataset_procesado_df = pd.read_parquet('dataset_final.parquet')

In [3]:
dataset_original_df.head(2)

,id,overview
0,862,"Led by Woody, Andy's toys live happily in his ..."
1,8844,When siblings Judy and Peter discover an encha...


In [4]:
dataset_procesado_df.head(2)

,budget,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,...,genre_name,company_name,company_id,country_iso_3166_1,country_name,language_iso_639_1,language_name,release_year,return,actor_name
0,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Animation,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."
1,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Comedy,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,..."


In [5]:
# Se convierten los id a enteros asegurando de que no haya valores nulos
dataset_original_df['id'] = pd.to_numeric(dataset_original_df['id'], errors='coerce').fillna(0).astype(int)
dataset_procesado_df['id'] = pd.to_numeric(dataset_procesado_df['id'], errors='coerce').fillna(0).astype(int)

In [6]:
movies_ml_df = pd.merge(dataset_procesado_df, dataset_original_df, on='id', how='left')

In [7]:
movies_ml_df.head(2)

,budget,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,...,company_name,company_id,country_iso_3166_1,country_name,language_iso_639_1,language_name,release_year,return,actor_name,overview
0,30000000.0,862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","Led by Woody, Andy's toys live happily in his ..."
1,30000000.0,862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","Led by Woody, Andy's toys live happily in his ..."


### **MODELO DE RECOMENDACIÓN**

#### Se crea un nuevo DataFrame conservando solo las columnas necesarias para el modelo, manteniendo la estructura de `movies_ml_df` intacta para futuras referencias o modificaciones.

In [8]:
columns_necesarias = ['title', 'genre_name', 'vote_average', 'popularity', 'overview', 'release_date']
model_df = movies_ml_df[columns_necesarias].copy()

In [9]:
model_df.head(10)

,title,genre_name,vote_average,popularity,overview,release_date
0,Toy Story,Animation,7.7,21.946943,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
1,Toy Story,Comedy,7.7,21.946943,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
2,Toy Story,Family,7.7,21.946943,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
3,Jumanji,Adventure,6.9,17.015539,When siblings Judy and Peter discover an encha...,1995-12-15
4,Jumanji,Adventure,6.9,17.015539,When siblings Judy and Peter discover an encha...,1995-12-15
5,Jumanji,Adventure,6.9,17.015539,When siblings Judy and Peter discover an encha...,1995-12-15
6,Jumanji,Adventure,6.9,17.015539,When siblings Judy and Peter discover an encha...,1995-12-15
7,Jumanji,Adventure,6.9,17.015539,When siblings Judy and Peter discover an encha...,1995-12-15
8,Jumanji,Adventure,6.9,17.015539,When siblings Judy and Peter discover an encha...,1995-12-15
9,Jumanji,Fantasy,6.9,17.015539,When siblings Judy and Peter discover an encha...,1995-12-15


In [10]:
model_df.shape

(383104, 6)

#### **Duplicados**
- #### Se eliminan los duplicados que se crearon al utilizar el metodo explode() durante el desanidado de la columna `genre` para no afectar la implementación del modelo de recomendación.

In [11]:
# Se liminan duplicados basados en el título de la película
model_df = model_df.drop_duplicates(subset=['title'], keep='first')

model_df.head(10)

,title,genre_name,vote_average,popularity,overview,release_date
0,Toy Story,Animation,7.7,21.946943,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
3,Jumanji,Adventure,6.9,17.015539,When siblings Judy and Peter discover an encha...,1995-12-15
21,Grumpier Old Men,Romance,6.5,11.712900,A family wedding reignites the ancient feud be...,1995-12-22
25,Waiting to Exhale,Comedy,6.1,3.859495,"Cheated on, mistreated and stepped on, the wom...",1995-12-22
28,Father of the Bride Part II,Comedy,5.7,8.387519,Just when George Banks has recovered from his ...,1995-02-10
30,Heat,Action,7.7,17.924927,"Obsessive master thief, Neil McCauley leads a ...",1995-12-15
54,Sabrina,Comedy,6.2,6.677277,An ugly duckling having undergone a remarkable...,1995-12-15
110,Tom and Huck,Action,5.4,2.561161,"A mischievous young boy, Tom Sawyer, witnesses...",1995-12-22
118,Sudden Death,Action,5.5,5.231580,International action superstar Jean Claude Van...,1995-12-22
127,GoldenEye,Adventure,6.6,14.686036,James Bond must unmask the mysterious head of ...,1995-11-16


In [12]:
model_df.shape

(42197, 6)

### **Preprocesamiento de texto de ``overview``**

- Eliminación de símbolos de puntuación
- Conversión de texto a minúsculas
- Eliminación de stopwords 

In [13]:
print(model_df['overview'].iloc[0])

Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


In [14]:
import re
import string
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text: str) -> str:
    """
    Preprocesa un texto eliminando puntuación, convirtiendo a minúsculas y eliminando stopwords.
    
    Parámetros:
    text (str): El texto original que se va a preprocesar.
    
    Retorna:
    str: El texto preprocesado.
    """
    # Se elimina puntuación y conversión a minúsculas
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    
    # Se remuven stopwords
    text = " ".join([word for word in text.split() if word not in stop_words])
    
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Se aplica el preprocesamiento a la columna 'overview'
model_df['overview'] = model_df['overview'].fillna("").apply(preprocess_text)


In [16]:
# resultado con la primer fila
print(model_df['overview'].iloc[0])

led woody andys toys live happily room andys birthday brings buzz lightyear onto scene afraid losing place andys heart woody plots buzz circumstances separate buzz woody owner duo eventually learns put aside differences


### **Preprocesamiento para la Similitud del Coseno**
#### Para la Similitud del Coseno se utiliza una muestra más reducida del dataset, tomando como criterio las distribuciones obtenidas en el EDA:
- Se realiza un filtro por fecha, solo tomando las películas que fueron lanzadas desde la década del 90.
- Se aplica un filtro teniendo en cuenta los votos (calificación promedio: 6)

In [17]:
model_df.shape

(42197, 6)

In [18]:
# películas posteriores a 1990
model_df = model_df[model_df['release_date'] >= '1990-01-01']
model_df.shape

(27613, 6)

In [19]:
# películas con un puntaje mayor o igual a 6
model_df = model_df[model_df['vote_average'] >= 6]
model_df.shape

(14434, 6)

In [20]:
model_df.head(5)

,title,genre_name,vote_average,popularity,overview,release_date
0,Toy Story,Animation,7.7,21.946943,led woody andys toys live happily room andys b...,1995-10-30
3,Jumanji,Adventure,6.9,17.015539,siblings judy peter discover enchanted board g...,1995-12-15
21,Grumpier Old Men,Romance,6.5,11.712900,family wedding reignites ancient feud nextdoor...,1995-12-22
25,Waiting to Exhale,Comedy,6.1,3.859495,cheated mistreated stepped women holding breat...,1995-12-22
30,Heat,Action,7.7,17.924927,obsessive master thief neil mccauley leads top...,1995-12-15


### **Vectorización con TF-IDF**
- Se convierten los textos a una representación numérica para que capture la relevancia de las palabras y poder calcular similitudes entre las películas basándonos en sus descripciones.

- Se reinician los índices del dataframe, ya que luego del filtrado quedan desajustados. Esto asegura que se encuentren alineados con la matriz TF-IDF y no ocasione inconvenientes.

In [21]:
# Reinicio de los índices 
model_df.reset_index(drop=True, inplace=True)

In [22]:
model_df.head(5)

,title,genre_name,vote_average,popularity,overview,release_date
0,Toy Story,Animation,7.7,21.946943,led woody andys toys live happily room andys b...,1995-10-30
1,Jumanji,Adventure,6.9,17.015539,siblings judy peter discover enchanted board g...,1995-12-15
2,Grumpier Old Men,Romance,6.5,11.712900,family wedding reignites ancient feud nextdoor...,1995-12-22
3,Waiting to Exhale,Comedy,6.1,3.859495,cheated mistreated stepped women holding breat...,1995-12-22
4,Heat,Action,7.7,17.924927,obsessive master thief neil mccauley leads top...,1995-12-15


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Conservamos solo las 5000 palabras mas frecuentes
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Se aplica el vectorizador a la columna 'overview'
tfidf_matrix = tfidf_vectorizer.fit_transform(model_df['overview'])

print(tfidf_matrix.shape)


(14434, 5000)


- Se comprueba que estén presentes palabras claves en la matriz TF-IDF

In [24]:
# Se obtienen las palabras en la matriz 
palabras_tfidf = tfidf_vectorizer.get_feature_names_out()

# Palabras a buscar
palabras_clave = ['love', 'war']

# Comprobación
for palabra in palabras_clave:
    if palabra in palabras_tfidf:
        print(f"'{palabra}' está presente en la matriz TF-IDF.")
    else:
        print(f"'{palabra}' NO está presente en la matriz TF-IDF.")


'love' está presente en la matriz TF-IDF.
'war' está presente en la matriz TF-IDF.


- Se comprueban las palabras más relevantes para la película Toy Story 

In [25]:
# Película de ejemplo
pelicula_ejemplo = model_df[model_df['title'] == 'Toy Story'].index[0]

# Se extraen las palabras relevantes y sus pesos en la matriz
palabras_relevantes = tfidf_matrix[pelicula_ejemplo].toarray()[0]
vocabulario = tfidf_vectorizer.get_feature_names_out()

# El resultado lo vemos en un diccionario
palabras_pesos = dict(zip(vocabulario, palabras_relevantes))

# Se muestran las palabras más relevantes de la película
palabras_mas_relevantes = {k: v for k, v in palabras_pesos.items() if v > 0}
print(sorted(palabras_mas_relevantes.items(), key=lambda x: x[1], reverse=True)[:10])



[('woody', np.float64(0.6238574127324098)), ('toys', np.float64(0.20481424788208774)), ('aside', np.float64(0.1971492178573003)), ('plots', np.float64(0.19304201166559232)), ('afraid', np.float64(0.18948418783251286)), ('differences', np.float64(0.18099927464440205)), ('happily', np.float64(0.18040050561656276)), ('onto', np.float64(0.1750538154571675)), ('separate', np.float64(0.1750538154571675)), ('duo', np.float64(0.1745737285873019))]


### **Cálculo de la Similitud del Coseno**

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

# Se calcula la similitud del coseno en la matriz TF-IDF
sim_coseno = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(f"Dimensiones: {sim_coseno.shape}")


Dimensiones: (14434, 14434)


### **Función de Recomendación**

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

def recomendacion(titulo: str, num_recomendaciones: int = 5) -> list:
    """
    Dada una película, devuelve una lista con los títulos de las películas más similares.

    Parámetros:
    - titulo (str): El título de la película para la cual se quieren recomendaciones.
    - num_recomendaciones (int): Número de recomendaciones a devolver. Por defecto es 5.

    Retorna:
    - list: Lista con los títulos de las películas recomendadas.
    """
    
    # Se verifica si está disponible la película
    if titulo not in model_df['title'].values:
        return ["El título no está en el dataset."]
    
    # Se obtiene el índice de la película 
    idx = model_df[model_df['title'] == titulo].index[0]
    
    # Se hace el calculo del coseno entre la película y todas las demás
    sim_scores = list(enumerate(sim_coseno[idx]))
    
    # Se ordenan las películas en base a la similitud (de mayor a menor)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Se obtiene los índices de las películas más similares, excluyendo la película original
    sim_scores = sim_scores[1:num_recomendaciones + 1]
    
    # Se obtiene las películas más similares
    movie_indices = [i[0] for i in sim_scores]
    
    return model_df['title'].iloc[movie_indices].tolist()

In [28]:
print(recomendacion('Spider-Man'))

['The Amazing Spider-Man', 'The Amazing Spider-Man 2', 'Spider-Man 2', 'Kick-Ass', 'Austin High']


### **EXPORTACIÓN DE DATASET**

##### Se exportan los datos filtrados por fecha (posterior a 1990) y puntuación (promedio de votos mayor o igual a 6). Este dataset contiene las columnas: ``title``, ``genre_name``, ``vote_average``, ``popularity``, ``overview``, y ``release_date``.



##### También se exporta la matriz TF-IDF y la matriz de Similitud. De esta forma se evita que la API la vuelva a generar, y solamente cumpla su responsabilidad principal: servir los datos.

In [250]:
model_df.head(5)

,title,genre_name,vote_average,popularity,overview,release_date
0,Toy Story,Animation,7.7,21.946943,led woody andys toys live happily room andys b...,1995-10-30
1,Jumanji,Adventure,6.9,17.015539,siblings judy peter discover enchanted board g...,1995-12-15
2,Grumpier Old Men,Romance,6.5,11.712900,family wedding reignites ancient feud nextdoor...,1995-12-22
3,Waiting to Exhale,Comedy,6.1,3.859495,cheated mistreated stepped women holding breat...,1995-12-22
4,Heat,Action,7.7,17.924927,obsessive master thief neil mccauley leads top...,1995-12-15


- Se exporta el dataframe utilizado para la implemetación del modelo en formato parquet

In [251]:
# Se guarda el DataFrame en formato parquet
model_df.to_parquet('model_ml.parquet', index=False)


- Se exporta la matriz TF-IDF en un archicvo .npz

In [29]:
from scipy import sparse

In [254]:
# Se guarda la matriz TF-IDF en un archivo .npz
sparse.save_npz('tfidf_matrix.npz', tfidf_matrix)

- Se exporta el vectorizador utilizado

In [255]:
import pickle

In [256]:
# Se guarda el vectorizador en un archivo .pkl
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

- Se exporta la matriz de similitud 

In [32]:
import numpy as np

In [34]:
# Se guarda la matriz de similitud en un archivo .npy (para matrices densas)
np.save('cosine_sim.npy', sim_coseno)